# Using Jurassic-2 Grande Instruct on SageMaker through Model Packages

## <span style='color:Red'>THIS MODEL HAS A NEW NAME</span>

### The model you are looking for is now called [Jurassic-2 Mid](https://github.com/AI21Labs/SageMaker/blob/main/J2_Mid_example_model_use.ipynb).